In [19]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

In [ ]:
driver: Chrome
options = Options()
options.add_argument("--lang=en")
options.add_argument("--headless-new")


def sign_in():
    driver.get("https://steamcommunity.com/login/home/")
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//input[@type="text" and not(@id)]'))
    ).send_keys("ursart")
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//input[@type="password"]'))
    ).send_keys("ADS250407")
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.XPATH, '//button[@type="submit" and contains(text(), "Sign in")]')
        )
    ).click()
    WebDriverWait(driver, 300).until(
        EC.url_to_be("https://steamcommunity.com/id/UrsArty")
    )


def get_current_page_items():
    return WebDriverWait(driver, 5).until(
        EC.presence_of_all_elements_located(
            (
                By.XPATH,
                '//div[not(contains(@style, "display: none;"))]/div[@class="itemHolder"]/div/a',
            )
        )
    )


def scrap_selected_item(item):
    ActionChains(driver).move_to_element(item).click().pause(0.25).perform()
    selected_item = driver.find_element(
        By.XPATH,
        '//div[@class="inventory_page_right"]/div[not(contains(@style, "display: none;"))]',
    )
    if "Trading Card" in selected_item.text:
        return False
    elif (
        selected_item.find_element(
            By.XPATH,
            './div[contains(@id, "scrap_content")]/div[contains(@id, "item_scrap_actions")]',
        ).get_attribute("style")
        == "display: none;"
    ):
        return False
    selected_item.find_element(
        By.XPATH,
        './div[contains(@id, "scrap_content")]/div/div/a[contains(@id, "item_scrap_link")]',
    ).click()
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located(
            (
                By.XPATH,
                '//div[@class="newmodal" and not(contains(@style, "display: none;"))]/div[3]/div/div[2]/div[contains(@data-panel, "autoFocus")]',
            )
        )
    ).click()
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located(
            (
                By.XPATH,
                '//div[@class="newmodal" and not(contains(@style, "display: none;"))]/div[3]/div/div[2]/div[contains(@data-panel, "autoFocus")]',
            )
        )
    ).click()
    return True


def iterate_page_items(items):
    for item in items:
        if scrap_selected_item(item):
            break
    if items == get_current_page_items():
        return
    else:
        iterate_page_items(get_current_page_items())


def iterate_pages():
    driver.get("https://steamcommunity.com/id/UrsArty/inventory")
    for _ in range(
        int(driver.find_element(By.ID, "pagecontrol_cur").text),
        int(driver.find_element(By.ID, "pagecontrol_max").text) + 1,
    ):
        if iterate_page_items(get_current_page_items()) is None:
            ActionChains(driver).move_to_element(
                driver.find_element(By.ID, "pagebtn_next")
            ).click().pause(0.25).perform()

In [ ]:
with Chrome(service=Service(), options=options) as driver:
    iterate_pages()